POS tagging is the task of assigning each word in a sentence with its corresponding Part-of-Speech (POS). POS tagging is a special type of classification task called sequence labelling which means that the POS tag is assigned to the words in a sequence by considering the POS of surrounding words. Therefore, this assignment requires some contextual information about that word. This can be achieved by algorithms which can remember the context. Earlier days, Hidden Markov Model (HMM) was used for modeling such data. But, now deep learning algorithms such as RNN and LSTM can effectively model the sequential data with the help of word embedding algorithms. Word embedding algorithmslike Word2vec represent words in a vectorof given dimension. It is assumed that this vector contains the semantics (meaning) of that particular word in the given document. This word2vec works by finding the similarity between the words in the document that means, it takes the context of each word into account before generating the embedding. 

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from keras import backend as K
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.


In [3]:
# This is the function to read data and to separate words and labels. Input to the function is path to the file

def get_data(path):
    texts = []
    labels = []
    read_file = open(path, 'r')
    for line in read_file:
        line = line.replace('\n','')
        if line == 'newline':            #???
            pass
        else:
            items = line.split('\t')
            texts.append(items[0])
            labels.append(items[1])
    read_file.close()

    return texts, labels

In [4]:
path = 'pos_data.txt'
words, tags = get_data(path)
words = [words]

In [5]:
# generate the word2vec model
wvmodel = Word2Vec(words, size=100, window=1, min_count=1)  #???

In [7]:
X = []
for i in range(len(words[0])):
    X.append(wvmodel.wv[words[0][i]])

In [8]:
m, n = np.shape(X)
X = np.reshape(X, (m,n,1))
print(np.shape(X))

(1048576, 100, 1)


In [17]:
np.shape(X.shape[1:])

(2,)

In [18]:
X[1:].shape

(1048575, 100, 1)

In [43]:
len(set(y))

42

In [9]:
# Converting the labels into integers
label_encoder = LabelEncoder()
label_encoder.fit(tags)
y = label_encoder.transform(tags)

The task is to develop a RNN or LSTM model for POS tagging in English. You are free to use any deep learning framework and archutecture. You can also try different methods for representing data other than word2vec

In [10]:
from keras.utils import to_categorical
y=to_categorical(y)


### Split the model into training and testing

In [12]:
x_train,x_test,y_train,y_test= train_test_split(X,y,test_size=0.30,random_state=0)

In [64]:
y.shape

(1048576, 42)

#### LSTM Model

In [13]:
model = Sequential()
#model.add(Embedding(100,100))
model.add(LSTM(32,activation='relu', input_shape = x_train.shape[1:]))
model.add(Dense(42,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_1 (Dense)              (None, 42)                1386      
Total params: 5,738
Trainable params: 5,738
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [16]:
model.fit(x_train,y_train,batch_size=32,epochs=1,verbose=1,validation_data=(x_test,y_test))

Train on 734003 samples, validate on 314573 samples
Epoch 1/1
734003/734003 [==============================] - 1411s 2ms/step - loss: 6.8724 - acc: 0.0312 - val_loss: 6.8896 - val_acc: 0.0313


In [21]:
score=model.evaluate(x_test,y_test,verbose=1)
print("Test accuracy = ",score[1]*100)

314573/314573 [==============================] - 105s 334us/step
Test accuracy =  3.130275007720705


### Simple RNN

In [18]:
model1 = Sequential()
#model.add(Embedding(100,100))
model1.add(SimpleRNN(32,activation='relu', input_shape = x_train.shape[1:]))
model1.add(Dense(1,activation='sigmoid'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 32)                1088      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,121
Trainable params: 1,121
Non-trainable params: 0
_________________________________________________________________


In [19]:
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [20]:
model.fit(x_train,y_train,batch_size=100,epochs=2,verbose=1,validation_data=(x_test,y_test))

Train on 734003 samples, validate on 314573 samples
Epoch 1/2
734003/734003 [==============================] - 636s 866us/step - loss: 6.8724 - acc: 0.0312 - val_loss: 6.8896 - val_acc: 0.0313
Epoch 2/2
734003/734003 [==============================] - 616s 839us/step - loss: 6.8724 - acc: 0.0312 - val_loss: 6.8896 - val_acc: 0.0313


In [22]:
score=model.evaluate(x_test,y_test,verbose=1)

314573/314573 [==============================] - 105s 335us/step
